In [1]:
# !python -m venv env
# !source env/bin/activate

In [2]:
!pip install transformers sentence-transformers
!pip install langchain
!pip install -U langchain-community
!pip install pinecone
!pip install boto3
!pip install openai
# !pip install einops
# !pip list

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the foll

In [6]:
import os
from sentence_transformers import SentenceTransformer
import numpy as np
# import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader


In [7]:

# # Load Sentence-BERT model from Hugging Face (free version)
# model = SentenceTransformer('jinaai/jina-embeddings-v2-small-en', trust_remote_code=True).cuda()  # Lightweight model for sentence embeddings

# model

In [9]:

# Function to load and chunk text from a file
def chunk_text_from_file(file_path, chunk_size=7600, chunk_overlap=500):
    loader = TextLoader(file_path)
    document = loader.load()
    
    # Split the document into chunks using RecursiveCharacterTextSplitter
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_documents(document)
    return chunks


In [15]:

# Function to generate embeddings for the text chunks
# def generate_embeddings(text_chunks):
#     embeddings = model.encode([chunk.page_content for chunk in text_chunks], convert_to_numpy=True)
#     return embeddings

from openai import OpenAI
client = OpenAI(api_key="",)

def get_embedding(text, model="text-embedding-3-small"):
    embeddings = []
    for chunk in text: 
        try:
            # print(chunk.page_content)
            chunk = chunk.page_content.replace("\n", " ")
            embeddings.append(client.embeddings.create(input = [chunk], model=model).data[0].embedding)
        except Exception as e:
            chunk = chunk.replace("\n", " ")
            embeddings.append(client.embeddings.create(input = [chunk], model=model).data[0].embedding)
    return embeddings


In [18]:
pinecone_chunks = []
# Store embeddings and paper IDs in FAISS index
def store_paper_embeddings_in_faiss(folder_path, txt_pdf):
    
    all_embeddings = []
    paper_ids = []
    pdf_metadata = []
    papers_uploaded = {}
    counter = 1
    for file_number, file_name in enumerate(os.listdir(folder_path)):
        print(counter)
        counter += 1
        # temp_dict = {}
        
        file_path = os.path.join(folder_path, file_name)
        
        
        if os.path.isfile(file_path) and file_name.endswith('.txt'):
            paper_id = "paper_" + str(txt_pdf[file_name[:-4]][0])
            chunks = chunk_text_from_file(file_path)
            embeddings = get_embedding(chunks)
            # print(len(embeddings))
            # print(len(embeddings[0]))
            # print(len(embeddings[1]))
            for i, value in enumerate(chunks):
                temp_dict = {}
                temp_dict["id"] = paper_id + "#chunk_" + str(i)
                temp_dict["values"] = embeddings[i]                
                temp_dict["metadata"] = {"paper_id": paper_id, "chunk_id": "chunk_" + str(i), "chunk": chunks[i].page_content}
                pinecone_chunks.append(temp_dict)

            all_embeddings.extend(embeddings)
            paper_ids.extend([paper_id] * len(embeddings))

    # Convert list to numpy array for FAISS
    embeddings_array = np.array(all_embeddings).astype('float32')

    return paper_ids


In [25]:
# Example usage
import json
folder_path = '/kaggle/input/textfiles-2001-end'  # Folder where your research papers (text files) are stored

with open('/kaggle/input/metadata-pdf/pdf_metadata.json', 'r') as file:
    txt_pdf = json.load(file)

paper_ids = store_paper_embeddings_in_faiss(folder_path,txt_pdf)
# print(faiss_index)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [26]:
len(pinecone_chunks)

10563

In [27]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_733LpQ_3vUPKxHKBvL21VL4JaZVMoW1XxF2xFyWFZDx5brAgnbrQ5UVoySp7ad26QU416D")
index = pc.Index("v2-research-paper-index")

In [28]:
def batch_upsert(index, vectors, batch_size=100):
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i+batch_size]
        index.upsert(vectors=batch)

In [29]:
batch_upsert(index, pinecone_chunks)
# index.upsert(vectors=pinecone_chunks)

In [93]:
# text = "What is the purpose of the PAC-Bayes theorem in machine learning, and how is it applied to classification problems?"
# query_embedding = get_embedding([text])

In [94]:
# response = index.query(
#     vector= query_embedding,
#     top_k=10,
#     include_metadata=True,
# )
# print(response)

In [32]:
4709+10563+4458

19730